In [1]:
## Aexiv-- Research
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [2]:
# Used the inbuilt tools wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki =WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
# Used the inbuilt tools Arxiv
api_wrapper_arxiv= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [5]:
# Custom tools[RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
EMBEDDING_MODEL = "nomic-embed-text"
OLLAMA_MODEL = "llama3.2"
embeddings= OllamaEmbeddings(model=EMBEDDING_MODEL)

In [7]:
loader = WebBaseLoader("https://docs.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200 ).split_documents(docs)
vectorstordb = FAISS.from_documents(documents, embedding=embeddings)

In [8]:
retriever = vectorstordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002735EAAEE50>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tools = create_retriever_tool(retriever, "langsmith-search", "search any information about Langsmith")

retriever_tools.name

'langsmith-search'

In [10]:
tools = [wiki, arxiv, retriever_tools]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data_science\\Gen AI\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002736FDBCF40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002735EAAEE5

In [12]:
## Run all this tools with Agents and LLM Models
model=ChatOllama(model=OLLAMA_MODEL)

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()



os.environ["LANGCHAIN_API_KEY"]  = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [14]:
#Prompt Template 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
system_prompt = (
            "You are an assistant for question-answering tasks."
            "Use the following pieces of retrieved context to answer"
            "the question. If you don't know, say that you don't know."
            "Use three sentences maximum and keep the answer concise." # Constraints on the LLM's response style
            
        )
prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history", optional=True), # Includes history for conversational continuity in the answer
                ("human", "{input}"), # Includes the original user question
                MessagesPlaceholder("agent_scratchpad")
            ]
        )
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.Use the following pieces of retrieved context to answerthe question. If you don't know, say that you don't know.Use three sentences maximum and keep the answer concise."), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(model, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [16]:
#Agent Excuter
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [17]:
agent_executor.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


LangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.Get startedBuild your first agentGet startedSign up for LangSmithTry LangSmithSend your first traceGet startedBuild an advanced agentGet startedEnroll in LangChain AcademyGet startedOpen source agent frameworks Python TypeScriptLangChain (Python)Quickly get started building agents, with any model provider of your choice.Learn moreLangGraph (Python)Control every step of your custom agent with low-level orchestration, memory, and human-in-the-loop support.Learn moreDeep AgentsBuild agents that can tackle complex, multi-step tasks.Learn moreLangSmithObservabilitySee exactly how your agent thinks and acts with detailed tracing and aggregate trend metrics.Learn moreEvaluationTest and score agent behavior on production data or offline datasets to continuously improve performance.Learn morePrompt Eng

{'input': 'Tell me about langsmith',
 'output': 'Langsmith is a deployment platform that helps AI teams use live production data for continuous testing and improvement. It provides observability tools, evaluation capabilities, and deployment features to ship agents in one click. Langsmith is now known as the LangGraph Platform.'}

In [18]:
agent_executor.invoke({"input": "What ia Machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witMachine learning is a field of study in artificial intelligence that involves developing algorithms to enable computers to learn from data, make decisions, and improve their performance on a specific task. It is based on the idea that machines can be trained to recognize patterns in data and make predictions or take actions without being explicitly programmed for each task.

> Finished chain.


{'input': 'What ia Machine learning',
 'output': 'Machine learning is a field of study in artificial intelligence that involves developing algorithms to enable computers to learn from data, make decisions, and improve their performance on a specific task. It is based on the idea that machines can be trained to recognize patterns in data and make predictions or take actions without being explicitly programmed for each task.'}

In [19]:
agent_executor.invoke({"input": "what is the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe paper "Attention Is All You Need" by Ashish Vaswani et al. (arXiv:1706.03762) introduces a new architecture for neural machine translation and other sequence-to-sequence tasks, which uses self-attention mechanisms to weigh the importance of different input elements when generating output. This approach is particularly effective in handling long-range dependencies and has since become a widely adopted standard in natural language processing. The paper was published in 2017 and revolutionized the field of NLP by demonstrating that attention mechanisms can be used to achieve state-of-the-art results in many tasks.

> Finished chain.


{'input': 'what is the paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" by Ashish Vaswani et al. (arXiv:1706.03762) introduces a new architecture for neural machine translation and other sequence-to-sequence tasks, which uses self-attention mechanisms to weigh the importance of different input elements when generating output. This approach is particularly effective in handling long-range dependencies and has since become a widely adopted standard in natural language processing. The paper was published in 2017 and revolutionized the field of NLP by demonstrating that attention mechanisms can be used to achieve state-of-the-art results in many tasks.'}